In [1]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Define Control File

In [3]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

## Read From Control File

In [4]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

## Paths

In [5]:
tiers_output_dir = os.path.join(ScenarioDir, "Performance_Metrics", "Tiered_Outcome_Measures", "Reservoir_FloodRisk", "Tiers")
tiers_output_path = os.path.join(tiers_output_dir, "tier_assignment.csv")
probs_output_dir = os.path.join(ScenarioDir, "Performance_Metrics", "Metrics", "Reservoir_FloodRisk")
probs_output_path = os.path.join(probs_output_dir, "probabilities_assignment.csv")

# check if output directory exists
if not os.path.exists(tiers_output_dir):
    # print warning
    print("Warning: directory " + tiers_output_dir + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(tiers_output_dir)
    
# check if output directory exists
if not os.path.exists(probs_output_dir):
    # print warning
    print("Warning: directory " + probs_output_dir + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(probs_output_dir)

## Read in Data

In [6]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df = add_water_year_column(df)

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:174: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


## Tier Assignments

#### Thresholds

In [7]:
tier1_tier2_thresh = 0.1
tier2_tier3_thresh = 0.4

#### Variables

In [8]:
# USING BOTH S_SLUIS_CVP_ and S_SLUIS_SWP_
# NOTE that S_MELON has only LEVEL4, and that S_MLRTN is a constant
# variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MELON_", "S_MLRTN_"]
# variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV", "S_MELON_LEVEL5DV"]
variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MLRTN_", "S_MELON_"]
variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL4DV", "S_SLUIS_CVPLEVEL4DV", "S_SLUIS_SWPLEVEL4DV", "S_MLRTNLEVEL4DV", "S_MELONLEVEL4DV"]

##### Helper Function

In [9]:
def classify_tier(prob, thresh1 = 0.10, thresh2 = 0.40):
    if np.isnan(prob):
        return np.nan
    elif prob < thresh1:
        return "1"
    elif prob < thresh2:
        return "2"
    else:
        return "3"

In [13]:
# hitting floodpool = variables_storage >= variables_floodpool
#FOLSOM is Nan for scenario 6 - 10 because it doesn't have level5D for sceanrio 6-10 "S_FOLSMLEVEL5DV"
S_MLRTN_level5constant = 524

probs_result_list = []
tiers_result_list = []

for scenario in dss_names:
    scenario_short = scenario[:5]
    probs_row_data = {"Scenario": scenario_short}
    tiers_row_data = {"Scenario": scenario_short}

    for var_s, var_lvl5D in zip(variables_storage, variables_floodpool):
        var1_name = f"{var_s}{scenario_short}"
        var2_name = f"{var_lvl5D}_{scenario_short}"

        p_gte = probability_var1_gte_var2_for_scenario(df, var1_name, var2_name, units="TAF", tolerance=1e-6)
        if(var_s == "S_MLRTN_"):
            p_gte = probability_var1_gte_const_for_scenario(df, var1_name, S_MLRTN_level5constant, units="TAF")
        
        gte_col = f"Prob_{var_s}GTE_{var_lvl5D}"
        tier_col = f"{var_s}FloodTier"

        probs_row_data[gte_col] = p_gte
        tiers_row_data[tier_col] = classify_tier(p_gte, tier1_tier2_thresh, tier2_tier3_thresh)

    probs_result_list.append(probs_row_data)
    tiers_result_list.append(tiers_row_data)


df_flow_probs = pd.DataFrame(probs_result_list)
df_flow_probs.set_index("Scenario", inplace=True)
df_tiers = pd.DataFrame(tiers_result_list)
df_tiers.set_index("Scenario", inplace=True)

display(df_flow_probs)
display(df_tiers)

,Prob_S_SHSTA_GTE_S_SHSTALEVEL5DV,Prob_S_OROVL_GTE_S_OROVLLEVEL5DV,Prob_S_FOLSM_GTE_S_FOLSMLEVEL5DV,Prob_S_TRNTY_GTE_S_TRNTYLEVEL4DV,Prob_S_SLUIS_CVP_GTE_S_SLUIS_CVPLEVEL4DV,Prob_S_SLUIS_SWP_GTE_S_SLUIS_SWPLEVEL4DV,Prob_S_MLRTN_GTE_S_MLRTNLEVEL4DV,Prob_S_MELON_GTE_S_MELONLEVEL4DV
Scenario,,,,,,,,
s0001,0.333333,0.215000,0.431667,0.182500,0.078333,0.065833,0.023333,0.149167
s0002,0.315000,0.210000,0.423333,0.174167,0.076667,0.073333,0.024167,0.163333
s0003,0.222500,0.165833,0.339167,0.110833,0.055000,0.060000,0.027500,0.120833
s0004,0.192500,0.144167,0.305000,0.092500,0.046667,0.051667,0.028333,0.110833
s0005,0.175833,0.133333,0.280000,0.080833,0.041667,0.042500,0.027500,0.100000
s0006,0.322695,0.216312,NaN,0.184397,0.090426,0.061170,0.045213,0.174645
s0007,0.320922,0.214539,NaN,0.190603,0.093085,0.170213,0.045213,0.174645
s0008,0.320035,0.212766,NaN,0.192376,0.093972,0.191489,0.045213,0.174645
s0009,0.320035,0.214539,NaN,0.193262,0.096631,0.201241,0.045213,0.174645


,S_SHSTA_FloodTier,S_OROVL_FloodTier,S_FOLSM_FloodTier,S_TRNTY_FloodTier,S_SLUIS_CVP_FloodTier,S_SLUIS_SWP_FloodTier,S_MLRTN_FloodTier,S_MELON_FloodTier
Scenario,,,,,,,,
s0001,2,2,3,2,1,1,1,2
s0002,2,2,3,2,1,1,1,2
s0003,2,2,2,2,1,1,1,2
s0004,2,2,2,1,1,1,1,2
s0005,2,2,2,1,1,1,1,2
s0006,2,2,NaN,2,1,1,1,2
s0007,2,2,NaN,2,1,2,1,2
s0008,2,2,NaN,2,1,2,1,2
s0009,2,2,NaN,2,1,2,1,2


In [14]:
## Save
    
df_flow_probs.to_csv(probs_output_path)
df_tiers.to_csv(tiers_output_path)

In [15]:
print("Done!")

Done!
